<a href="https://colab.research.google.com/github/zaelcovsky/workshop_deep_python_autumn_2023_not_my_group/blob/main/deep_python_vk_workshop_mapreduce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MapReduce: Word Count

В данном задании предлагается реализовать простейший MapReduce над корпусом текстов — нужно выполнить предобработку текстов и подсчет количества вхождений слов (и реализовать многопроцессно!)

In [ ]:
# !pip install scikit-learn nltk

In [ ]:
import re

from typing import List

import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords as nltk_stop
nltk.download("stopwords")

from sklearn.datasets import fetch_20newsgroups

[nltk_data] Downloading package stopwords to /home/dmitry/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Из текстов будем убирать стоп-слова — очень часто встречаемые в языке слова (артикли, предлоги, местоимения)

In [ ]:
stopwords = set(nltk_stop.words("english"))
list(stopwords)[:5]

['by', 'up', 'both', 'they', 'where']

### Скачивание текстов

In [ ]:
news = fetch_20newsgroups().data * 10
len(news)

113140

### Подсчет в одном процессе

In [ ]:
from collections import Counter

In [ ]:
def tokenize(text: str) -> List[str]:
    tokenizer = RegexpTokenizer(r"\w+")
    return tokenizer.tokenize(text)


def clear_tokens(tokens: List[str]) -> List[str]:
    return [re.sub(r"[^a-z]", "", token) for token in tokens]


def is_token_informative(token: str) -> bool:
    return token and token not in stopwords and len(token) > 2


def count_words(texts: List[str]):
    tokenizer = RegexpTokenizer(r"\w+")
    cnt = Counter()
    for text in texts:
        tokens = tokenize(text.lower())
        tokens_clear = clear_tokens(tokens)
        informative_tokens = filter(is_token_informative, tokens_clear)
        cnt.update(informative_tokens)
    return cnt

In [ ]:
%%time
cnt = count_words(news)
cnt.most_common(10)

CPU times: user 31.8 s, sys: 13.5 ms, total: 31.8 s
Wall time: 31.8 s


[('edu', 213210),
 ('com', 122880),
 ('subject', 122650),
 ('lines', 118370),
 ('organization', 112330),
 ('one', 90150),
 ('would', 89100),
 ('writes', 78440),
 ('article', 74380),
 ('people', 59760)]

### Подсчет с помощью MapReduce

Задание: реализация MapReduce с помощью многопроцессности

**Note:** Пригодится functools.reduce